# CH. 4 - TOPIC MODELS
## Activities

#### Activity 1

In [1]:
# not necessary
# added to suppress warnings coming from pyLDAvis

import warnings
warnings.filterwarnings('ignore')

In [2]:
import langdetect  # language detection
import matplotlib.pyplot  # plotting
import nltk  # natural language processing
import numpy  # arrays and matrices
import pandas  # dataframes
import pyLDAvis  # plotting
import pyLDAvis.sklearn  # plotting
import regex  # regular expressions
import sklearn  # machine learning

In [5]:
# define path

path = '~/Documents/packt-data/topic-model-health-tweets/latimeshealth.txt'

In [6]:
# load data

df = pandas.read_csv(path, sep="|", header=None)
df.columns = ["id", "datetime", "tweettext"]

In [7]:
# define quick look function for data frame

def dataframe_quick_look(df, nrows):
    print("SHAPE:\n{shape}\n".format(shape=df.shape))
    print("COLUMN NAMES:\n{names}\n".format(names=df.columns))
    print("HEAD:\n{head}\n".format(head=df.head(nrows)))

In [8]:
dataframe_quick_look(df, nrows=2)

SHAPE:
(4171, 3)

COLUMN NAMES:
Index(['id', 'datetime', 'tweettext'], dtype='object')

HEAD:
                   id                        datetime  \
0  576760256031682561  Sat Mar 14 15:02:15 +0000 2015   
1  576715414811471872  Sat Mar 14 12:04:04 +0000 2015   

                                           tweettext  
0  Five new running shoes that aim to go the extr...  
1  Gym Rat: Disq class at Crunch is intense worko...  



In [9]:
# view final data that will be carried forward

raw = df['tweettext'].tolist()
print("HEADLINES:\n{lines}\n".format(lines=raw[:5]))
print("LENGTH:\n{length}\n".format(length=len(raw)))

HEADLINES:
['Five new running shoes that aim to go the extra mile http://lat.ms/1ELp3wU', 'Gym Rat: Disq class at Crunch is intense workout on pulley system http://lat.ms/1EKOFdr', 'Noshing through thousands of ideas at Natural Products Expo West http://lat.ms/1EHqywg', 'Natural Products Expo also explores beauty, supplements and more http://lat.ms/1EHqyfE', 'Free Fitness Weekends in South Bay beach cities aim to spark activity http://lat.ms/1EH3SMC']

LENGTH:
4171



In [10]:
# define function for checking language of tweets
# filter to english only

def do_language_identifying(txt):
    try:
       the_language = langdetect.detect(txt)
    except:
       the_language = 'none'
    return the_language

In [11]:
# define function to perform lemmatization

def do_lemmatizing(wrd):
    out = nltk.corpus.wordnet.morphy(wrd)
    return (wrd if out is None else out)

In [12]:
# define function to cleaning tweet data

def do_tweet_cleaning(txt):
    # identify language of tweet
    # return null if language not english
    lg = do_language_identifying(txt)
    if lg != 'en':
        return None
    
    # split the string on whitespace
    out = txt.split(' ')
    
    # identify screen names
    # replace with SCREENNAME
    out = ['SCREENNAME' if i.startswith('@') else i for i in out]
    
    # identify urls
    # replace with URL
    out = [
        'URL' if bool(regex.search('http[s]?://', i)) 
        else i for i in out
    ]
    
    # remove all punctuation
    out = [regex.sub('[^\\w\\s]|\n', '', i) for i in out]
    
    # make all non-keywords lowercase
    keys = ['SCREENNAME', 'URL']
    out = [i.lower() if i not in keys else i for i in out]
    
    # remove keywords
    out = [i for i in out if i not in keys]
    
    # remove stopwords
    list_stop_words = nltk.corpus.stopwords.words('english')
    list_stop_words = [regex.sub('[^\\w\\s]', '', i) for i in list_stop_words]
    
    out = [i for i in out if i not in list_stop_words]
    
    # lemmatizing
    out = [do_lemmatizing(i) for i in out]
    
    # keep words 4 or more characters long
    out = [i for i in out if len(i) >= 5]
    
    return out

In [13]:
# apply cleaning function to every tweet

clean = list(map(do_tweet_cleaning, raw))

In [14]:
# remove none types

clean = list(filter(None.__ne__, clean))
print("HEADLINES:\n{lines}\n".format(lines=clean[:5]))
print("LENGTH:\n{length}\n".format(length=len(clean)))

HEADLINES:
[['running', 'shoes', 'extra'], ['class', 'crunch', 'intense', 'workout', 'pulley', 'system'], ['thousand', 'natural', 'product'], ['natural', 'product', 'explore', 'beauty', 'supplement'], ['fitness', 'weekend', 'south', 'beach', 'spark', 'activity']]

LENGTH:
4093



In [15]:
# turn tokens back into strings
# concatenate using whitespaces

clean_sentences = [" ".join(i) for i in clean]

In [16]:
print(clean_sentences[0:10])

['running shoes extra', 'class crunch intense workout pulley system', 'thousand natural product', 'natural product explore beauty supplement', 'fitness weekend south beach spark activity', 'kayla harrison sacrifice', 'sonic treatment alzheimers disease', 'ultrasound brain restore memory alzheimers needle onlyso farin mouse', 'apple researchkit really medical research', 'warning chantix drink taking might remember']


#### Activity 2

In [17]:
# define global variables

number_words = 10
number_docs = 10
number_features = 1000

In [18]:
# bag of words conversion
# count vectorizer (raw counts)

vectorizer1 = sklearn.feature_extraction.text.CountVectorizer(
    analyzer="word",
    max_df=0.95, 
    min_df=10, 
    max_features=number_features
)
clean_vec1 = vectorizer1.fit_transform(clean_sentences)
print(clean_vec1[0])

feature_names_vec1 = vectorizer1.get_feature_names()

  (0, 321)	1


In [19]:
# define function to calculate perplexity based on number of topics

def perplexity_by_ntopic(data, ntopics):
    output_dict = {
        "Number Of Topics": [], 
        "Perplexity Score": []
    }
    
    for t in ntopics:
        lda = sklearn.decomposition.LatentDirichletAllocation(
            n_components=t,
            learning_method="online",
            random_state=0
        )
        lda.fit(data)
        
        output_dict["Number Of Topics"].append(t)
        output_dict["Perplexity Score"].append(lda.perplexity(data))
        
    output_df = pandas.DataFrame(output_dict)
    
    index_min_perplexity = output_df["Perplexity Score"].idxmin()
    output_num_topics = output_df.loc[
        index_min_perplexity,  # index
        "Number Of Topics"  # column
    ]
        
    return (output_df, output_num_topics)

In [20]:
# execute function on vector of numbers of topics
# takes several minutes

df_perplexity, optimal_num_topics = perplexity_by_ntopic(
    clean_vec1, 
    ntopics=[i for i in range(1, 21) if i % 2 == 0]
)

In [21]:
print(df_perplexity)

   Number Of Topics  Perplexity Score
0                 2        352.529190
1                 4        406.377785
2                 6        435.641900
3                 8        456.371887
4                10        481.002936
5                12        486.622771
6                14        510.467910
7                16        507.923273
8                18        540.964044
9                20        534.081098


In [22]:
# define and fit lda model

lda = sklearn.decomposition.LatentDirichletAllocation(
    n_components=optimal_num_topics,
    learning_method="online",
    random_state=0
)
lda.fit(clean_vec1)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [23]:
# define function to format raw output into nice tables

def get_topics(mod, vec, names, docs, ndocs, nwords):
    # word to topic matrix
    W = mod.components_
    W_norm = W / W.sum(axis=1)[:, numpy.newaxis]
    # topic to document matrix
    H = mod.transform(vec)
    
    W_dict = {}
    H_dict = {}
    
    for tpc_idx, tpc_val in enumerate(W_norm):
        topic = "Topic{}".format(tpc_idx)
        
        # formatting w
        W_indices = tpc_val.argsort()[::-1][:nwords]
        W_names_values = [
            (round(tpc_val[j], 4), names[j]) 
            for j in W_indices
        ]
        W_dict[topic] = W_names_values
        
        # formatting h
        H_indices = H[:, tpc_idx].argsort()[::-1][:ndocs]
        H_names_values = [
            (round(H[:, tpc_idx][j], 4), docs[j]) 
            for j in H_indices
        ]
        H_dict[topic] = H_names_values
        
    W_df = pandas.DataFrame(
        W_dict, 
        index=["Word" + str(i) for i in range(nwords)]
    )
    H_df = pandas.DataFrame(
        H_dict,
        index=["Doc" + str(i) for i in range(ndocs)]
    )
        
    return (W_df, H_df)

In [24]:
# get nice tables

W_df, H_df = get_topics(
    mod=lda,
    vec=clean_vec1,
    names=feature_names_vec1,
    docs=raw,
    ndocs=number_docs, 
    nwords=number_words
)

In [25]:
# word-topic table

print(W_df)

                     Topic0                Topic1
Word0        (0.052, study)      (0.0385, latfit)
Word1       (0.0215, could)       (0.0327, study)
Word2       (0.0203, brain)      (0.0317, health)
Word3      (0.0168, report)       (0.029, cancer)
Word4   (0.0157, scientist)      (0.0226, people)
Word5      (0.0153, weight)  (0.0167, researcher)
Word6  (0.0151, california)       (0.0167, woman)
Word7     (0.0132, medical)     (0.0165, patient)
Word8    (0.0124, research)       (0.0165, death)
Word9    (0.011, treatment)      (0.0162, doctor)


In [26]:
# document-topic table

print(H_df)

                                                 Topic0  \
Doc0  (0.9373, RT @skgire: For my non-science friend...   
Doc1  (0.9355, Hospital counselors used free medicat...   
Doc2  (0.9351, Flu shot benefit for adults isn't big...   
Doc3  (0.9349, @Dr_Rocamboli Yes, that's what Steve ...   
Doc4  (0.9348, Expert panel from @RANDCorporation, @...   
Doc5  (0.9297, '5-a-day' might not be enough to ward...   
Doc6  (0.9294, Do you feel happier these days? World...   
Doc7  (0.9283, The @US_FDA is looking out for your p...   
Doc8  (0.9283, U.S. to accelerate clinical trials fo...   
Doc9  (0.9261, RT @mmacvean: Story from @theatlantic...   

                                                 Topic1  
Doc0  (0.94, Researchers use a safe version of HIV t...  
Doc1  (0.9386, Doctors often delay vaccines for youn...  
Doc2  (0.9372, From the Dept. of Happy Accidental Di...  
Doc3  (0.9372, Does your dog know how you're feeling...  
Doc4  (0.9371, RT @annagorman: Ok flats it is! “@was...  
Do

In [27]:
# iteractive plot
# pca biplot and histogram

lda_plot = pyLDAvis.sklearn.prepare(lda, clean_vec1, vectorizer1, R=10)
pyLDAvis.display(lda_plot)

#### Activity 3

In [28]:
# bag of words conversion
# tf-idf method

vectorizer2 = sklearn.feature_extraction.text.TfidfVectorizer(
    analyzer="word",
    max_df=0.5, 
    min_df=20, 
    max_features=number_features,
    smooth_idf=False
)
clean_vec2 = vectorizer2.fit_transform(clean_sentences)
print(clean_vec2[0])

feature_names_vec2 = vectorizer2.get_feature_names()

In [29]:
# define and fit nmf model

nmf = sklearn.decomposition.NMF(
    n_components=optimal_num_topics,
    init="nndsvda",
    solver="mu",
    beta_loss="frobenius",
    random_state=0, 
    alpha=0.1, 
    l1_ratio=0.5
)
nmf.fit(clean_vec2)

NMF(alpha=0.1, beta_loss='frobenius', init='nndsvda', l1_ratio=0.5,
    max_iter=200, n_components=2, random_state=0, shuffle=False, solver='mu',
    tol=0.0001, verbose=0)

In [30]:
# get nicely formatted result tables

W_df, H_df = get_topics(
    mod=nmf,
    vec=clean_vec2,
    names=feature_names_vec2,
    docs=raw,
    ndocs=number_docs, 
    nwords=number_words
)

In [31]:
# word-topic table

print(W_df)

                  Topic0                Topic1
Word0    (0.3765, study)      (0.5972, latfit)
Word1   (0.0257, cancer)       (0.0486, steps)
Word2   (0.0207, people)       (0.0449, today)
Word3  (0.0184, obesity)    (0.0372, exercise)
Word4   (0.0184, health)  (0.0274, healthtips)
Word5    (0.0181, brain)     (0.0257, workout)
Word6  (0.0173, suggest)     (0.0204, getting)
Word7   (0.0168, weight)     (0.0192, fitness)
Word8    (0.0153, woman)       (0.0143, great)
Word9    (0.0131, death)     (0.0132, morning)


In [32]:
# document-topic table

print(H_df)

                                                 Topic0  \
Doc0  (0.2032, Today's small step for health: Add a ...   
Doc1  (0.2032, Race, education linked to differences...   
Doc2  (0.2032, Drug czar says addiction is a health ...   
Doc3  (0.2032, FDA approves first drug treatment for...   
Doc4  (0.2032, It's not too late to get your flu sho...   
Doc5  (0.2032, RT @montemorin: Just say 'NO' to Noog...   
Doc6  (0.2032, FDA's removal of restrictions on sale...   
Doc7  (0.2032, Legalizing gay marriage is good for p...   
Doc8  (0.2032, Reverse vaccine for Type 1 diabetes p...   
Doc9  (0.2032, Warning: Shaking hands with your doct...   

                                                 Topic1  
Doc0  (0.2277, Fruit flies take one for the (footbal...  
Doc1  (0.2277, How many steps do you take each day? ...  
Doc2  (0.2277, Rate of repeat births among teen moth...  
Doc3  (0.2277, Undisclosed soy prompts recall by Lis...  
Doc4  (0.2277, Supplements we take mostly not ordere...  
Do